In [ ]:
from datetime import datetime
import calendar
import arcpy
import os

# Base URL for Minnesota GIS data
BASE_URL = "https://gisdata.mn.gov/api/3/action/"

FOLDER_PATH = "GIS5572"
DESKTOP_PATH = os.path.join(os.path.expanduser("~"), "Desktop")
DATASET_DIRECTORY = os.path.join(DESKTOP_PATH, FOLDER_PATH)


# FUNCTION TO PULL DATA FROM MNGIS STORE IT IN THE HOME FOLDER
def pullFromMNGIS(datasetName: str):
    SAVE_DIR = os.path.join(DESKTOP_PATH, FOLDER_PATH, datasetName)
    os.makedirs(SAVE_DIR, exist_ok=True)

    # Fetch dataset details
    response = requests.get(f"{BASE_URL}package_show", params={"id": datasetName})
    data = response.json()

    if data["success"]:
        resources = data["result"]["resources"]

        # Find the TIFF or ZIP resource
        tiff_url = None
        for resource in resources:
            if "tif" in resource["url"].lower() or "zip" in resource["url"].lower():
                tiff_url = resource["url"]
                file_name = os.path.join(SAVE_DIR, os.path.basename(tiff_url))
                print(f"Downloading: {tiff_url}")

                # Download the file
                with requests.get(tiff_url, stream=True) as r:
                    r.raise_for_status()
                    with open(file_name, "wb") as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            f.write(chunk)
                print(f"Downloaded: {file_name}")

                # Unzip if it's a ZIP file
                if file_name.endswith(".zip"):
                    with zipfile.ZipFile(file_name, "r") as zip_ref:
                        zip_ref.extractall(SAVE_DIR)
                    print(f"Extracted: {SAVE_DIR}")

            else:
                print("Failed to fetch dataset information.")

pullFromMNGIS("biota-bmsb")

# Path to your GeoPackage DATASET
BMSB_DATASET_PATH = os.path.join(DATASET_DIRECTORY, "biota-bmsb")
gpkg_path = os.path.join(BMSB_DATASET_PATH, "BMSBSurveyDataTable.dbf")

BMSB_OUTPUT_PATH = os.path.join(DATASET_DIRECTORY, "OUTPUT", "bmsb_data.csv")

aprx = arcpy.mp.ArcGISProject("CURRENT")
map_obj = aprx.listMaps()[0]  # Assuming you want to add to the first map

map_obj.addDataFromPath(gpkg_path)

# CONVERT TO XY POINTS
arcpy.management.XYTableToPoint(
    in_table="BMSBSurveyDataTable",
    out_feature_class=r"BMSBSurveyData_XY_Points",
    x_field="Longitude",
    y_field="Latitude",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)

# ADDS CTU DATA TO XY POINTS
arcpy.analysis.SpatialJoin(
    target_features="BMSBSurveyData_XY_Points",
    join_features="ctus_with_id",
    out_feature_class=r"BMSBSurveyData_SpatialJoin",
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='SiteName "SiteName" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,SiteName,0,254;SiteType "SiteType" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,SiteType,0,254;City "City" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,City,0,254;County "County" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,County,0,254;Latitude "Latitude" true true false 8 Double 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Latitude,-1,-1;Longitude "Longitude" true true false 8 Double 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Longitude,-1,-1;SurveyName "SurveyName" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,SurveyName,0,254;Scientific "Scientific" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Scientific,0,254;CommonName "CommonName" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,CommonName,0,254;Surveyor "Surveyor" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Surveyor,0,254;Year "Year" true true false 8 Double 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Year,-1,-1;TrapID "TrapID" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,TrapID,0,254;TrapType "TrapType" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,TrapType,0,254;Lure "Lure" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Lure,0,254;CheckDate "CheckDate" true true false 8 Date 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,CheckDate,-1,-1;Adults "Adults" true true false 8 Double 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Adults,-1,-1;Nymphs "Nymphs" true true false 8 Double 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Nymphs,-1,-1;Notes "Notes" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Notes,0,254;GNIS_FEATU "GNIS_FEATU" true true false 10 Long 0 10,First,#,ctus_with_id,GNIS_FEATU,-1,-1;FEATURE_NA "FEATURE_NA" true true false 254 Text 0 0,First,#,ctus_with_id,FEATURE_NA,0,254;CTU_CLASS "CTU_CLASS" true true false 25 Text 0 0,First,#,ctus_with_id,CTU_CLASS,0,25;COUNTY_GNI "COUNTY_GNI" true true false 10 Long 0 10,First,#,ctus_with_id,COUNTY_GNI,-1,-1;COUNTY_COD "COUNTY_COD" true true false 2 Text 0 0,First,#,ctus_with_id,COUNTY_COD,0,2;COUNTY_NAM "COUNTY_NAM" true true false 100 Text 0 0,First,#,ctus_with_id,COUNTY_NAM,0,100;POPULATION "POPULATION" true true false 10 Long 0 10,First,#,ctus_with_id,POPULATION,-1,-1;SHAPE_Leng "SHAPE_Leng" true true false 19 Double 0 0,First,#,ctus_with_id,SHAPE_Leng,-1,-1;SHAPE_Area "SHAPE_Area" true true false 19 Double 0 0,First,#,ctus_with_id,SHAPE_Area,-1,-1;ORIG_FID "ORIG_FID" true true false 10 Long 0 10,First,#,ctus_with_id,ORIG_FID,-1,-1;UNIQUE_ID "UNIQUE_ID" true true false 10 Long 0 10,First,#,ctus_with_id,UNIQUE_ID,-1,-1',
    match_option="WITHIN",
    search_radius=None,
    distance_field_name=""
)

# CLIPS OUT ANYTHING OUTSIDE OF MN BOUNDARIES
arcpy.analysis.PairwiseClip(
    in_features="BMSBSurveyData_SpatialJoin",
    clip_features=r"BOUNDARIES OF MINNESOTA",
    out_feature_class=r"BMSBSurveyData_PairwiseClip",
    cluster_tolerance=None
)

# ADDS FIELD FOR PRESENCE OF BMSB
arcpy.management.AddField(
    in_table="BMSBSurveyData_PairwiseClip",
    field_name="BMSBPresence",
    field_type="SHORT",
    field_precision=None,
    field_scale=None,
    field_length=None,
    field_alias="",
    field_is_nullable="NULLABLE",
    field_is_required="NON_REQUIRED",
    field_domain=""
)

# ADDS FIELD TO PUT IN MONTH
arcpy.management.AddField(
    in_table="BMSBSurveyData_PairwiseClip",
    field_name="Month",
    field_type="SHORT",
    field_precision=None,
    field_scale=None,
    field_length=None,
    field_alias="",
    field_is_nullable="NULLABLE",
    field_is_required="NON_REQUIRED",
    field_domain=""
)

# Iterate through and clean rows
with arcpy.da.UpdateCursor(r"BMSBSurveyData_PairwiseClip", ['Year', 'SiteType', 'Latitude', 'Longitude', 'Adults', 'Nymphs', 'Notes', 'CheckDate','BMSBPresence','Month']) as cursor:  # Use OBJECTID or another unique identifier
    # Iterate through the rows
    for row in cursor:
        # Replace this condition with your logic to identify rows to delete
        if ( not ((row[0] <= 2025) and (row[0] >= 2000) and (row[4] >= 0) and (row[4] <= 1000000) and (row[5] >= 0) and (row[5] <= 1000000) and (row[6] != """Gate Locked""") and (row[6] != """Gate Locked - didn't check"""))):
            # BAD ROW. QAQC BAD DATA REMOVAL
            cursor.deleteRow()   
        else:
            # SETS MONTH AND BMSB PRESENCE
            cursor.updateRow(row[:8]+[int((row[4]+row[5])>0),row[7].month])

# EXPORT FEATURE CLASS TO CSV
BMSB_OUTPUT_PATH = os.path.join(DATASET_DIRECTORY, "OUTPUT", "bmsb_data.csv")
os.mkdir(os.path.join(DATASET_DIRECTORY, "OUTPUT"))
arcpy.conversion.ExportTable(r"BMSBSurveyData_PairwiseClip", BMSB_OUTPUT_PATH)